In [1]:
import torch
import model
from tokenizer import BytePairTokenizer, TikToken4o

byte_training/tok.json
loaded a model with  4201 vocabulary!
target vocab size is:  None !
Displaying the sequence task!
Decoded Inputs:
<|unknown|>Title: S
Title: Sim
itle: Simple
le: Simple S

Decoded Targets:
Title: Sim
itle: Simple
le: Simple S
: Simple Sab
Encoded output shape: torch.Size([4, 6, 3])


In [7]:
phrase = 'every journey begins with a single step'

tokenizer = TikToken4o
attention = model.self_attention_layer

tokens = tokenizer.encode(phrase)
embeddings = model.embedding_layer(torch.tensor(tokens))
context = attention(embeddings)
context.detach().numpy()

array([[-0.39175302, -0.21735917,  0.8390444 ],
       [ 0.02595548,  0.02246953,  0.792957  ],
       [-0.2733143 , -0.14544621,  0.8325386 ],
       [-0.0709865 , -0.03826859,  0.8048437 ],
       [-0.03547822, -0.06003229,  0.78393734],
       [-0.06488673, -0.07848479,  0.78802633],
       [-0.0111661 , -0.08959781,  0.7641735 ]], dtype=float32)